# Préparation des données

In [1]:
import numpy as np
import pandas as pd
import warnings

# Athlètes

In [2]:
athlete = pd.read_json('data/olympic_athletes.json')
athlete.head()

athlete_url                 0
athlete_full_name           0
games_participations        0
first_game                 22
athlete_year_birth       2456
athlete_medals          60552
bio                     53062
dtype: int64

# Hôtes

In [3]:
hosts = pd.read_xml('data/olympic_hosts.xml')
hosts.head()
hosts.drop(['index'], axis=1, inplace=True)

game_slug          0
game_end_date      0
game_start_date    0
game_location      0
game_name          0
game_season        0
game_year          0
dtype: int64

# Médailles

In [4]:
medals = pd.read_excel('data/olympic_medals.xlsx')
medals.shape
medals = medals.rename({'slug_game': 'game_slug'}, axis=1)
medals.loc[(medals.duplicated())]
medals.loc[(medals.game_slug == 'paris-1900') & (medals.discipline_title == 'Polo')]
medals.loc[(medals.game_slug == 'london-1908') & (medals.discipline_title == 'Polo')]
medals.loc[(medals.game_slug == 'london-1908') & (medals.discipline_title == 'Hockey') & (medals.event_title == 'hockey men')]
medals.drop(['Unnamed: 0', 'participant_title', 'athlete_url'], axis=1, inplace=True)
medals.tail()

,discipline_title,game_slug,event_title,event_gender,medal_type,participant_type,athlete_full_name,country_name,country_code,country_3_letter_code
21692,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,SILVER,Athlete,Viggo JENSEN,Denmark,DK,DEN
21693,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,BRONZE,Athlete,Alexandros Nikolopoulos,Greece,GR,GRE
21694,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,GOLD,Athlete,Viggo JENSEN,Denmark,DK,DEN
21695,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,SILVER,Athlete,Launceston ELLIOT,Great Britain,GB,GBR
21696,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,BRONZE,Athlete,Sotirios VERSIS,Greece,GR,GRE


# Résultats

In [5]:
results = pd.read_html('data/olympic_results.html')[0]
results.head()
results.shape
results.loc[(results.duplicated())]
results.drop(['Unnamed: 0', 'athlete_url'], axis=1, inplace=True)
results = results.rename({'slug_game': 'game_slug'}, axis=1)

Index(['discipline_title', 'event_title', 'game_slug', 'participant_type',
       'medal_type', 'athletes', 'rank_equal', 'rank_position', 'country_name',
       'country_code', 'country_3_letter_code', 'athlete_full_name',
       'value_unit', 'value_type'],
      dtype='object')

# Séparation JO d'Hiver & JO d'Eté

In [6]:
jo_types = hosts[['game_slug', 'game_season', 'game_year']]
merged_hosts_results = results.merge(jo_types, on='game_slug')
merged_hosts_results.game_season.unique()

array(['Winter', 'Summer'], dtype=object)

In [7]:
SG_results = merged_hosts_results.loc[(merged_hosts_results.game_season == 'Summer')].copy()
winter_games_results = merged_hosts_results.loc[(merged_hosts_results.game_season == 'Winter')].copy()

In [8]:
SG_results.drop(['game_season'], axis=1, inplace=True)
dopplers = SG_results.loc[(SG_results.duplicated())]
dopplers

,discipline_title,event_title,game_slug,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_full_name,value_unit,value_type,game_year
154567,Water Polo,Water Polo Women,antwerp-1920,GameTeam,NaN,NaN,NaN,1,Netherlands,NL,NED,NaN,2,SCORE,1920
155542,Sailing,8m mixed,stockholm-1912,GameTeam,NaN,NaN,NaN,5,Russian Federation,RU,RUS,NaN,0,POINTS,1912
157220,Shooting,trap 125 targets men,stockholm-1912,Athlete,GOLD,NaN,NaN,1,United States of America,US,USA,Jay Graham,96,POINTS,1912
157221,Shooting,trap 125 targets men,stockholm-1912,Athlete,SILVER,NaN,NaN,2,Germany,DE,GER,Alfred GOELDEL,94,POINTS,1912
157222,Shooting,trap 125 targets men,stockholm-1912,Athlete,BRONZE,NaN,NaN,3,Russian Federation,RU,RUS,Harry Blaus,91,POINTS,1912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161942,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161943,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161944,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161945,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900


# Calcul du nombre total de médailles, pour chaque type, par pays et par Jeux

In [9]:

SG_results['medal_type'] = SG_results['medal_type'].fillna('None')
SG_results['total_medals'] = SG_results['medal_type'].apply(lambda x: 0 if x == 'None' else 1)
SG_results['gold_medals'] = SG_results['medal_type'].apply(lambda x: 1 if x == 'GOLD' else 0)
SG_results['silver_medals'] = SG_results['medal_type'].apply(lambda x: 1 if x == 'SILVER' else 0)
SG_results['bronze_medals'] = SG_results['medal_type'].apply(lambda x: 1 if x == 'BRONZE' else 0)
hist_MedalsbCountry = SG_results.groupby(['game_year', 'country_name']).agg({'total_medals': 'sum', 'gold_medals': 'sum', 'silver_medals': 'sum', 'bronze_medals': 'sum'})
hist_MedalsbCountry = hist_MedalsbCountry.sort_values(by=['game_year', 'total_medals'],ascending=[True, False]).reset_index()
hist_MedalsbCountry

,game_year,country_name,total_medals,gold_medals,silver_medals,bronze_medals
0,1896,Greece,47,10,18,19
1,1896,United States of America,20,11,7,2
2,1896,Germany,13,6,5,2
3,1896,France,11,5,4,2
4,1896,Great Britain,7,2,3,2
...,...,...,...,...,...,...
2884,2020,"Virgin Islands, British",0,0,0,0
2885,2020,"Virgin Islands, US",0,0,0,0
2886,2020,Yemen,0,0,0,0
2887,2020,Zambia,0,0,0,0


# Calcul du nombre total d'épreuve par pays, par Jeux

In [10]:
hist_SportsbCountry = SG_results.groupby(['game_year', 'country_name', 'discipline_title']).count()
hist_SportsbCountry = hist_SportsbCountry[['event_title']].reset_index()
hist_SportsbCountry = hist_SportsbCountry.rename({'discipline_title': 'sports', 'event_title': 'epreuves'},axis=1)
hist_SportsbCountry = hist_SportsbCountry.groupby(['game_year', 'country_name']).agg({'sports': 'count'})
hist_SportsbCountry = hist_SportsbCountry.reset_index()

hist_StagebCountry = SG_results.groupby(['game_year', 'country_name', 'event_title']).count()
hist_StagebCountry = hist_StagebCountry[['discipline_title']].reset_index()
hist_StagebCountry = hist_StagebCountry.rename({'event_title': 'epreuves', 'discipline_title': 'participation'}, axis=1)
hist_StagebCountry = hist_StagebCountry.groupby(['game_year', 'country_name']).agg({'epreuves': 'count'})
hist_StagebCountry = hist_StagebCountry.reset_index()

hist_olympData = hist_MedalsbCountry.merge(hist_SportsbCountry, on=['game_year', 'country_name'])
hist_olympData = hist_olympData.merge(hist_StagebCountry, on=['game_year', 'country_name'])
hist_olympData = hist_olympData.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])
hist_olympData

,game_year,country_name,total_medals,gold_medals,silver_medals,bronze_medals,sports,epreuves
0,1896,Greece,47,10,18,19,10,39
1,1896,United States of America,20,11,7,2,3,17
2,1896,Germany,13,6,5,2,7,27
3,1896,France,11,5,4,2,7,20
4,1896,Great Britain,7,2,3,2,9,20
...,...,...,...,...,...,...,...,...
2884,2020,"Virgin Islands, British",0,0,0,0,1,2
2885,2020,"Virgin Islands, US",0,0,0,0,1,1
2886,2020,Yemen,0,0,0,0,2,2
2887,2020,Zambia,0,0,0,0,3,5


In [11]:
# Utilisation de SG_results pour créer des colonnes qui représentent le nombre de participation au temps t, et le nombre de médailles gagnées au temps t-1 pour chaque pays et chaque jeu, fusionner les résultats avec hist_olympData

game_part = SG_results.groupby(['country_name', 'game_year']).agg(
    {'total_medals': 'sum', 'gold_medals': 'sum', 'silver_medals': 'sum', 'bronze_medals': 'sum'})
game_p = game_part.reset_index()
game_played_france = game_p.loc[(game_p.country_name == 'France')]

warnings.filterwarnings('ignore')
game_played_france = game_p.loc[(game_p.country_name == 'France')]
game_played_france['game_part'] = range(0, game_played_france.shape[0])
game_played_france['prec_game_medal'] = game_played_france['total_medals'].shift(1, fill_value=0)
game_played_france['prec_game_gold'] = game_played_france['gold_medals'].shift(1, fill_value=0)
game_played_france['prec_game_silver'] = game_played_france['silver_medals'].shift(1, fill_value=0)
game_played_france['prec_game_bronze'] = game_played_france['bronze_medals'].shift(1, fill_value=0)

liste_country = list(game_p.country_name.unique())
liste_country.remove('France')
liste_country
for country in liste_country:
    game_p_temp = game_p.loc[(game_p.country_name == country)]
    game_p_temp['game_part'] = range(0, game_p_temp.shape[0])
    game_p_temp['prec_game_medal'] = game_p_temp['total_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_gold'] = game_p_temp['gold_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_silver'] = game_p_temp['silver_medals'].shift(1, fill_value=0)
    game_p_temp['prec_game_bronze'] = game_p_temp['bronze_medals'].shift(1, fill_value=0)
    game_played_france = pd.concat([game_played_france, game_p_temp])

game_played_france = game_played_france[
    ['game_year', 'country_name', 'total_medals', 'gold_medals', 'silver_medals', 'bronze_medals', 'game_part',
     'prec_game_medal', 'prec_game_gold', 'prec_game_silver', 'prec_game_bronze']]
game_played_france = game_played_france.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])

game_played_france = game_played_france[['game_year', 'country_name', 'game_part', 'prec_game_medal', 'prec_game_gold', 'prec_game_silver',
     'prec_game_bronze']]

hist_olympData = pd.merge(hist_olympData, game_played_france, on=['game_year', 'country_name'])

hist_olympData = hist_olympData.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])

# Sauvegarde des données propres dans un CSV

In [12]:
hist_olympData.to_csv('data/olympic_data_cleaned.csv', index=False)

## Entrainement DNN

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras import models, layers

In [32]:
# Chargement des données
hist_olympData = pd.read_csv('data/olympic_data_cleaned.csv')

# Séparation des données avant et après 2020
bf2020 = hist_olympData[hist_olympData['game_year'] < 2020]
jo2020 = hist_olympData[hist_olympData['game_year'] == 2020]

# Définition des features et de la target
features = ['sports', 'epreuves', 'game_part', 'prec_game_medal', 'prec_game_gold', 'prec_game_silver', 'prec_game_bronze']
target = ['gold_medals', 'silver_medals', 'bronze_medals']

# Séparation des features et de la target
X_train = bf2020[features]
y_train = bf2020[target]
X_test = jo2020[features]
y_test = jo2020[target]
country = jo2020['country_name']

# Standardisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Construction du modèle
model_dense = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(3) # Couche de sortie avec 3 neurones pour prédire les médailles d'or, d'argent et de bronze
])

# Compilation du modèle
model_dense.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrainer le modèle
history_dense = model_dense.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluation du modèle
loss_dense, mae_dense = model_dense.evaluate(X_test_scaled, y_test, verbose=0)

# Prédiction des médailles pour 2020
y_pred_dense = model_dense.predict(X_test_scaled)

# Calcul des métriques
mse_dense = mean_squared_error(y_test, y_pred_dense)
mae_dense = mean_absolute_error(y_test, y_pred_dense)
r2_dense = r2_score(y_test, y_pred_dense)

print(f'Dense Neural Network - Mean Squared Error: {mse_dense}')
print(f'Dense Neural Network - Mean Absolute Error: {mae_dense}')
print(f'Dense Neural Network - R-squared: {r2_dense}')

# Fusion des résultats réels et prédits
results_dense = pd.DataFrame({
    'Country': country,
    'Actual Gold': y_test['gold_medals'],
    'Actual Silver': y_test['silver_medals'],
    'Actual Bronze': y_test['bronze_medals'],
    'Predicted Gold': y_pred_dense[:, 0],
    'Predicted Silver': y_pred_dense[:, 1],
    'Predicted Bronze': y_pred_dense[:, 2]
})

print(results_dense)

Epoch 1/100
68/68 [==============================] - 1s 3ms/step - loss: 28.8279 - mae: 1.8560 - val_loss: 5.1249 - val_mae: 0.9796
Epoch 2/100
68/68 [==============================] - 0s 2ms/step - loss: 12.0216 - mae: 1.2972 - val_loss: 2.1291 - val_mae: 0.6368
Epoch 3/100
68/68 [==============================] - 0s 2ms/step - loss: 10.8642 - mae: 1.1581 - val_loss: 2.0739 - val_mae: 0.6065
Epoch 4/100
68/68 [==============================] - 0s 2ms/step - loss: 10.4694 - mae: 1.1629 - val_loss: 2.1123 - val_mae: 0.6191
Epoch 5/100
68/68 [==============================] - 0s 2ms/step - loss: 9.5577 - mae: 1.1362 - val_loss: 2.1875 - val_mae: 0.6443
Epoch 6/100
68/68 [==============================] - 0s 2ms/step - loss: 9.2682 - mae: 1.1111 - val_loss: 2.2034 - val_mae: 0.6600
Epoch 7/100
68/68 [==============================] - 0s 3ms/step - loss: 8.8023 - mae: 1.1189 - val_loss: 2.2047 - val_mae: 0.6784
Epoch 8/100
68/68 [==============================] - 0s 2ms/step - loss: 8.6468

In [33]:
# Exporter le model au format h5 + le scaler au format pkl
model_dense.save('model/olympic_medal_prediction.h5')
import joblib
joblib.dump(scaler, 'model/olympic_medal_prediction_scaler.pkl')

# Prédiction pour les JO de Paris 2024

In [22]:
# Préparation des données pour la prédiction de 2024
jo2024 = jo2020.copy()
jo2024['game_year'] = 2024

# Utilisation des données de 2020 pour prédire 2024
jo2024['prec_game_medal'] = jo2020['total_medals']
jo2024['prec_game_gold'] = jo2020['gold_medals']
jo2024['prec_game_silver'] = jo2020['silver_medals']
jo2024['prec_game_bronze'] = jo2020['bronze_medals']

# Extraction des features pour 2024
X_2024 = jo2024[features]

# Standardisation des features
X_2024_scaled = scaler.transform(X_2024)

# Prediction des médailles pour 2024
y_pred_2024 = model_dense.predict(X_2024_scaled)

# Résultats de la prédiction
results_2024 = pd.DataFrame({
    'Country': jo2024['country_name'],
    'Predicted Gold': np.abs(np.round(y_pred_2024[:, 0])),
    'Predicted Silver': np.abs(np.round(y_pred_2024[:, 1])),
    'Predicted Bronze': np.abs(np.round(y_pred_2024[:, 2]))
})

print(results_2024)

6/6 [==============================] - 0s 1ms/step
                         Country  Predicted Gold  Predicted Silver  \
2699    United States of America            33.0              28.0   
2700  People's Republic of China            39.0              30.0   
2701                         ROC            25.0              23.0   
2702               Great Britain            13.0              13.0   
2703                       Japan            14.0              12.0   
...                          ...             ...               ...   
2884     Virgin Islands, British             0.0               0.0   
2885          Virgin Islands, US             0.0               0.0   
2886                       Yemen             0.0               0.0   
2887                      Zambia             0.0               0.0   
2888                    Zimbabwe             0.0               0.0   

      Predicted Bronze  
2699              29.0  
2700              29.0  
2701              25.0  
2702    

# Sauvegarde des résultats Paris 2024 dans un fichier CSV

In [30]:
results_2024.to_csv('data/olympic_medal_predictions2024.csv', index=False)

# Entrainement CNN

In [15]:
# Refonte des données pour la couche Conv1D
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Construction du modèle
model_cnn = models.Sequential([
    layers.Conv1D(32, 2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(3)
])

# Compilation du modèle
model_cnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrainement du modèle
history_cnn = model_cnn.fit(X_train_cnn, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluation du modèle
loss_cnn, mae_cnn = model_cnn.evaluate(X_test_cnn, y_test, verbose=0)

# Prediction des médailles pour 2020
y_pred_cnn = model_cnn.predict(X_test_cnn)

# Calcul des métriques
mse_cnn = mean_squared_error(y_test, y_pred_cnn)
mae_cnn = mean_absolute_error(y_test, y_pred_cnn)
r2_cnn = r2_score(y_test, y_pred_cnn)

print(f'CNN - Mean Squared Error: {mse_cnn}')
print(f'CNN - Mean Absolute Error: {mae_cnn}')
print(f'CNN - R-squared: {r2_cnn}')

# Fusion des résultats réels et prédits
results_cnn = pd.DataFrame({
    'Country': country,
    'Actual Gold': y_test['gold_medals'],
    'Actual Silver': y_test['silver_medals'],
    'Actual Bronze': y_test['bronze_medals'],
    'Predicted Gold': y_pred_cnn[:, 0],
    'Predicted Silver': y_pred_cnn[:, 1],
    'Predicted Bronze': y_pred_cnn[:, 2]
})

print(results_cnn)

Epoch 1/100
68/68 [==============================] - 1s 5ms/step - loss: 21.8137 - mae: 1.6926 - val_loss: 2.4472 - val_mae: 0.7468
Epoch 2/100
68/68 [==============================] - 0s 4ms/step - loss: 10.9140 - mae: 1.1753 - val_loss: 2.4163 - val_mae: 0.6876
Epoch 3/100
68/68 [==============================] - 0s 4ms/step - loss: 9.8856 - mae: 1.1512 - val_loss: 2.0269 - val_mae: 0.6311
Epoch 4/100
68/68 [==============================] - 0s 3ms/step - loss: 9.3704 - mae: 1.1288 - val_loss: 2.0609 - val_mae: 0.6976
Epoch 5/100
68/68 [==============================] - 0s 4ms/step - loss: 9.1065 - mae: 1.1170 - val_loss: 2.4284 - val_mae: 0.6612
Epoch 6/100
68/68 [==============================] - 0s 3ms/step - loss: 8.8706 - mae: 1.1004 - val_loss: 2.8329 - val_mae: 0.7892
Epoch 7/100
68/68 [==============================] - 0s 4ms/step - loss: 8.7032 - mae: 1.0880 - val_loss: 2.3230 - val_mae: 0.6133
Epoch 8/100
68/68 [==============================] - 0s 3ms/step - loss: 8.3663 -

# Entrainement RNN

In [16]:
# Remodeler les données pour la couche SimpleRNN
X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_rnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Construction du modèle
model_rnn = models.Sequential([
    layers.SimpleRNN(32, activation='relu', input_shape=(X_train_rnn.shape[1], 1)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(3)
])

# Compilation du modèle
model_rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrainement du modèle
history_rnn = model_rnn.fit(X_train_rnn, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluation du modèle
loss_rnn, mae_rnn = model_rnn.evaluate(X_test_rnn, y_test, verbose=0)

# Prediction des médailles pour 2020
y_pred_rnn = model_rnn.predict(X_test_rnn)

# Calcul des métriques
mse_rnn = mean_squared_error(y_test, y_pred_rnn)
mae_rnn = mean_absolute_error(y_test, y_pred_rnn)
r2_rnn = r2_score(y_test, y_pred_rnn)

print(f'RNN - Mean Squared Error: {mse_rnn}')
print(f'RNN - Mean Absolute Error: {mae_rnn}')
print(f'RNN - R-squared: {r2_rnn}')

# Fusion des résultats réels et prédits
results_rnn = pd.DataFrame({
    'Country': country,
    'Actual Gold': y_test['gold_medals'],
    'Actual Silver': y_test['silver_medals'],
    'Actual Bronze': y_test['bronze_medals'],
    'Predicted Gold': y_pred_rnn[:, 0],
    'Predicted Silver': y_pred_rnn[:, 1],
    'Predicted Bronze': y_pred_rnn[:, 2]
})

print(results_rnn)

Epoch 1/100
68/68 [==============================] - 2s 8ms/step - loss: 24.8953 - mae: 1.8098 - val_loss: 5.4378 - val_mae: 1.3156
Epoch 2/100
68/68 [==============================] - 0s 4ms/step - loss: 12.5994 - mae: 1.3531 - val_loss: 2.5745 - val_mae: 0.7945
Epoch 3/100
68/68 [==============================] - 0s 5ms/step - loss: 11.5014 - mae: 1.2541 - val_loss: 2.2147 - val_mae: 0.7329
Epoch 4/100
68/68 [==============================] - 0s 4ms/step - loss: 11.0474 - mae: 1.2207 - val_loss: 2.7638 - val_mae: 0.7639
Epoch 5/100
68/68 [==============================] - 0s 4ms/step - loss: 10.0355 - mae: 1.1327 - val_loss: 2.5069 - val_mae: 0.7142
Epoch 6/100
68/68 [==============================] - 0s 4ms/step - loss: 9.1013 - mae: 1.1245 - val_loss: 2.2848 - val_mae: 0.6235
Epoch 7/100
68/68 [==============================] - 0s 4ms/step - loss: 8.8260 - mae: 1.1048 - val_loss: 3.9105 - val_mae: 0.8138
Epoch 8/100
68/68 [==============================] - 0s 4ms/step - loss: 8.685